In [ ]:
#!docker run fastdotai/fastai2:latest python -c "from fastai2.vision.all import *"

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
ws

In [ ]:
%%writefile train.py
from fastai2.vision.all import *

path = untar_data(URLs.PETS)
path.ls()

files = get_image_files(path/"images")
len(files)

#(Path('/home/ashwin/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_102.jpg'),Path('/home/ashwin/.fastai/data/oxford-iiit-pet/images/great_pyrenees_102.jpg'))

def label_func(f): return f[0].isupper()

#To get our data ready for a model, we need to put it in a DataLoaders object. Here we have a function that labels using the file names, so we will use ImageDataLoaders.from_name_func. There are other factory methods of ImageDataLoaders that could be more suitable for your problem, so make sure to check them all in vision.data.

dls = ImageDataLoaders.from_name_func(path, files, label_func, item_tfms=Resize(224))

#We have passed to this function the directory we're working in, the files we grabbed, our label_func and one last piece as item_tfms: this is a Transform applied on all items of our dataset that will resize each imge to 224 by 224, by using a random crop on the largest dimension to make it a square, then resizing to 224 by 224. If we didn't pass this, we would get an error later as it would be impossible to batch the items together.

dls.show_batch()

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

In [ ]:
from azureml.core import Environment

fastai_env = Environment("fastai2")
fastai_env.docker.enabled = True
fastai_env.docker.base_image = "fastdotai/fastai2:latest"
fastai_env.python.user_managed_dependencies = True

In [ ]:
from azureml.core import ScriptRunConfig, Experiment 

fastai_config = ScriptRunConfig(source_directory='.', script='train.py')
fastai_config.run_config.environment = fastai_env
fastai_config.run_config.target = 'gpu-cluster'

In [ ]:
from azureml.core import Experiment

run = Experiment(ws,'fastai-custom-image').submit(fastai_config)
run

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

In [ ]:
import os 

os.remove('train.py')